# Data Cleaning Request

"Hey,

The board is asking to see how volume looked in Q2. I got some data (attached), but didn’t have a chance to pull anything together and was hoping you could take a stab at it.

I think they just want to see Q2 2021 volume by region and wanted to know if everything was looking good. I think this file has what you need. I don’t remember all the region codes – I know NAM ends in 1, EMEA ends in 3 and APAC and LATAM are 2 and 4, but I don’t remember which is which. I do know LATAM has the lowest volume so just go ahead and assign that to which ever comes out lowest.

I appreciate your help!"						

In [1]:
import pandas as pd
import numpy as np

# Import

In [2]:
df = pd.read_excel('Excel_for_Analytics_Project_Series_Source_File.xlsx', sheet_name ='EXT0070122021' )
geo = pd.read_excel('Excel_for_Analytics_Project_Series_Source_File.xlsx', sheet_name = 'Sheet3')

# Exploration

In [3]:
df.head()

,CLID,Date,Vol
0,CL11420,03/31/2020,884
1,NaN,04/30/2020,886
2,NaN,05/31/2020,968
3,NaN,06/30/2020,564
4,NaN,07/31/2020,648


In [4]:
geo.head()

,CLID,GEOID
0,C-CL69323,GEO1001
1,C-CL97995,GEO1001
2,C-CL87299,GEO1003
3,C-CL38496,GEO1001
4,C-CL75562,GEO1003


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 0 to 906
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CLID    53 non-null     object
 1   Date    907 non-null    object
 2   Vol     907 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.4+ KB


In [6]:
df['Date'].min()

'01/31/2020'

In [7]:
df['Date'].max()

'12/31/2020'

# Cleaning

In [8]:
#fill CLID down where there are blinks
df['CLID'].fillna(method = 'ffill', inplace = True)

#convert to datetime
df['Date'] = pd.to_datetime(df['Date'])

#add year column
df['Year'] = df['Date'].dt.year

#add quarter column
df['Quarter'] = df['Date'].dt.quarter

#add year quarter column
df['YearQuarter'] = df['Year'].astype(str) + '_' + df['Quarter'].astype(str)

In [9]:
df.dtypes

CLID                   object
Date           datetime64[ns]
Vol                     int64
Year                    int64
Quarter                 int64
YearQuarter            object
dtype: object

In [10]:
df.head(10)

,CLID,Date,Vol,Year,Quarter,YearQuarter
0,CL11420,2020-03-31,884,2020,1,2020_1
1,CL11420,2020-04-30,886,2020,2,2020_2
2,CL11420,2020-05-31,968,2020,2,2020_2
3,CL11420,2020-06-30,564,2020,2,2020_2
4,CL11420,2020-07-31,648,2020,3,2020_3
5,CL11420,2020-08-31,406,2020,3,2020_3
6,CL11420,2020-09-30,569,2020,3,2020_3
7,CL11420,2020-10-31,487,2020,4,2020_4
8,CL11420,2020-11-30,729,2020,4,2020_4
9,CL11420,2020-12-31,565,2020,4,2020_4


## Geo Data Clean

In [11]:
geo.head()

,CLID,GEOID
0,C-CL69323,GEO1001
1,C-CL97995,GEO1001
2,C-CL87299,GEO1003
3,C-CL38496,GEO1001
4,C-CL75562,GEO1003


In [12]:
# remove C- in front of CLID so it can be referenced correctly
geo['CLID'] = geo['CLID'].str.replace('C-', '')

#create dicationary and map CLID in df
geo_dict = geo.copy()
geo_dict.index = geo['CLID']
geo_dict = geo_dict['GEOID'].to_dict()

## Approach 1 (using merge)

In [13]:
# try diff approach with merge

In [19]:
dfgeo_merge =  df.merge(geo, on = 'CLID')
dfgeo_merge

,CLID,Date,Vol,Year,Quarter,YearQuarter,GEOID
0,CL11420,2020-03-31,884,2020,1,2020_1,GEO1004
1,CL11420,2020-04-30,886,2020,2,2020_2,GEO1004
2,CL11420,2020-05-31,968,2020,2,2020_2,GEO1004
3,CL11420,2020-06-30,564,2020,2,2020_2,GEO1004
4,CL11420,2020-07-31,648,2020,3,2020_3,GEO1004
...,...,...,...,...,...,...,...
902,CL99768,2021-05-31,290,2021,2,2021_2,GEO1002
903,CL99768,2021-04-30,294,2021,2,2021_2,GEO1002
904,CL99768,2021-03-31,270,2021,1,2021_1,GEO1002
905,CL99768,2021-02-28,224,2021,1,2021_1,GEO1002


In [26]:
dfgeo_merge['GEOID'].value_counts()

GEO1001    343
GEO1002    241
GEO1004    180
GEO1003    143
Name: GEOID, dtype: int64

# Approach 2 (using dictionary)

In [21]:
#map to df using the dictionary using CLID 
df['GEOID'] = df['CLID'].map(geo_dict)

In [22]:
df

,CLID,Date,Vol,Year,Quarter,YearQuarter,GEOID
0,CL11420,2020-03-31,884,2020,1,2020_1,GEO1004
1,CL11420,2020-04-30,886,2020,2,2020_2,GEO1004
2,CL11420,2020-05-31,968,2020,2,2020_2,GEO1004
3,CL11420,2020-06-30,564,2020,2,2020_2,GEO1004
4,CL11420,2020-07-31,648,2020,3,2020_3,GEO1004
...,...,...,...,...,...,...,...
902,CL99768,2021-05-31,290,2021,2,2021_2,GEO1002
903,CL99768,2021-04-30,294,2021,2,2021_2,GEO1002
904,CL99768,2021-03-31,270,2021,1,2021_1,GEO1002
905,CL99768,2021-02-28,224,2021,1,2021_1,GEO1002


In [28]:
df['GEOID'].value_counts()

GEO1001    343
GEO1002    241
GEO1004    180
GEO1003    143
Name: GEOID, dtype: int64

# Match GEOID with region code

In [31]:
#segement data to only quarter 2 of 2021
q2 = df[df['YearQuarter'] == '2021_2']

#look at volume of each geoID
q2vol = q2.groupby('GEOID')['Vol'].sum()
q2vol

GEOID
GEO1001    596502
GEO1002    109811
GEO1003    176338
GEO1004     82631
Name: Vol, dtype: int64

In [32]:
#create dictionary based on the email request
region_dic = {
    'GEO1001' : 'NAM',
    'GEO1002' : 'APAC', 
    'GEO1003' : 'EMEA',
    'GEO1004' : 'LATAM'
    
}

#map geoid to new column call Region 

df['Region'] = df['GEOID'].map(region_dic)


In [33]:
df

,CLID,Date,Vol,Year,Quarter,YearQuarter,GEOID,Region
0,CL11420,2020-03-31,884,2020,1,2020_1,GEO1004,LATAM
1,CL11420,2020-04-30,886,2020,2,2020_2,GEO1004,LATAM
2,CL11420,2020-05-31,968,2020,2,2020_2,GEO1004,LATAM
3,CL11420,2020-06-30,564,2020,2,2020_2,GEO1004,LATAM
4,CL11420,2020-07-31,648,2020,3,2020_3,GEO1004,LATAM
...,...,...,...,...,...,...,...,...
902,CL99768,2021-05-31,290,2021,2,2021_2,GEO1002,APAC
903,CL99768,2021-04-30,294,2021,2,2021_2,GEO1002,APAC
904,CL99768,2021-03-31,270,2021,1,2021_1,GEO1002,APAC
905,CL99768,2021-02-28,224,2021,1,2021_1,GEO1002,APAC
